In [1]:
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
structures = pq.ParquetFile('../data/structures.parquet').read(use_pandas_metadata=True).to_pandas(categories=['atom'])
train = pq.ParquetFile('../data/train.parquet').read(use_pandas_metadata=True).to_pandas(categories=['type'])
test = pq.ParquetFile('../data/test.parquet').read(use_pandas_metadata=True).to_pandas(categories=['type'])

/home/gautham/apps/anaconda3/envs/kaggle/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/gautham/apps/anaconda3/envs/kaggle/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/gautham/apps/anaconda3/envs/kaggle/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [3]:
train_mean = train.groupby('type')[['scalar_coupling_constant']].mean()

In [4]:
train_mean.loc[test.type].scalar_coupling_constant.head()

type
2JHC    -0.270624
1JHC    94.976151
3JHH     4.771023
1JHC    94.976151
2JHC    -0.270624
Name: scalar_coupling_constant, dtype: float32

In [5]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC


scalar_coupling_constant = np.empty(len(test), dtype='float32')
for i, row in test.iterrows():
    scalar_coupling_constant[i] = train_mean.loc[row.type].scalar_coupling_constant

In [6]:
result = train_mean.loc[test.type]

In [7]:
result_array = result.scalar_coupling_constant.array
result_series = pd.Series(result_array, test.index)

In [26]:
submission = test[['id']].copy()
submission['scalar_coupling_constant'] = result_series

In [29]:
submission.to_csv('simple_mean.csv', index=False)